In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
from remote_sensing_tools.raster_base import RasterBase, QueryParams, LoadParams
from remote_sensing_tools.stac_configuration import de_australia_stac_config
from remote_sensing_tools.band_indices import calculate_indices

## Load data
### Set up query and load parameters

In [ ]:
# set up AOI and time period
aus_query_params = QueryParams(
    bbox=(
        144.97,
        -37.47,
        144.95,
        -37.45,
    ),
    start_date="2020-01-01",
    end_date="2020-06-30",
)
# bands and res. used in .id for medians of MSAVI, BAEI, BSI
aus_load_params = LoadParams(
    crs="EPSG:3577",
    resolution=10,
    bands=("blue", "green", "red", "nir", "swir_1", "swir_2", "fmask"),
)

### Set up data configuration

In [ ]:
deaus_config = de_australia_stac_config

### Load using class method

In [ ]:
# query data from DE Aus
deaus_stac_raster = RasterBase.from_stac_query(
    config=deaus_config,
    collections=["ga_s2am_ard_3"],
    query_params=aus_query_params,
    load_params=aus_load_params,
)

deaus_stac_data = deaus_stac_raster.data
deaus_stac_data

## Calculate indicies using EO for .ID method

In this section, we calculate MSAVI, BAEI, BSI and NDWI indices for our AOI using the `calculate_indices` function from the `band_indices.py` in the Remote Sensing Tools folder. Once these indices are calculated, the median values for MSAVI, BAEI and BSI are calcluated and then the maximum values are calculated for NDWI. This analysis uses the same method used in the EO for .ID `analyse_aoi.py`.

In [ ]:
data_indices = calculate_indices(deaus_stac_data, ["ndwi", "bsi", "msavi", "baei"])
data_indices

In [ ]:
# calc. medians
ds_medians = (
    data_indices[["msavi", "baei", "bsi"]]
    .median(dim="time")
    .assign_coords(time=aus_query_params.end_date)
).compute()
ds_medians

In [ ]:
# calc. the max NDWI
ds_ndwi_max = (
    data_indices["ndwi"].max(dim="time").assign_coords(time=aus_query_params.end_date)
)

In [ ]:
# combine medians and max into single xarray
ds_combined = ds_medians.merge(ds_ndwi_max)

In [ ]:
# mask out all zero pixels
ds_masked_nonzero = ds_combined.where(
    (ds_combined["msavi"] != 0)
    | (ds_combined["baei"] != 0)
    | (ds_combined["bsi"] != 0)
    | (ds_combined["ndwi"] != 0)
).compute()
ds_masked_nonzero

### Plot indicies 

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

ds_masked_nonzero["ndwi"].plot.imshow(ax=axs[0, 0])
axs[0, 0].set_title("NDWI")
axs[0, 0].set_aspect("equal", adjustable="box")

ds_masked_nonzero["bsi"].plot.imshow(ax=axs[0, 1])
axs[0, 1].set_title("BSI")
axs[0, 1].set_aspect("equal", adjustable="box")

ds_masked_nonzero["msavi"].plot.imshow(ax=axs[1, 0])
axs[1, 0].set_title("MSAVI")
axs[1, 0].set_aspect("equal", adjustable="box")

ds_masked_nonzero["baei"].plot.imshow(ax=axs[1, 1])
axs[1, 1].set_title("BAEI")
axs[1, 1].set_aspect("equal", adjustable="box")

plt.tight_layout()
plt.show()